# Chat With Your Data

## Perform question answering

# Install libraries

In [ ]:
pip install openai

In [ ]:
pip install python-dotenv

In [ ]:
pip install langchain

In [ ]:
pip install langchain-openai

In [ ]:
pip install pypdf

In [ ]:
pip install faiss-cpu

In [ ]:
pip install langchainhub

## Load OpenAI API Key

In [50]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

## Prompt model with no knowledge of the Voynich manuscript

In [51]:
from langchain_openai import ChatOpenAI

#initialize the LLM we'll use - OpenAI GPT 3.5 Turbo
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")

In [52]:
#prompt the model with no additional knowledge of the Voynich manuscript beyond pretraining 
llm.invoke("What are the medicinal insights from the Voynich manuscript?")  

AIMessage(content='The Voynich manuscript is a mysterious and undeciphered text that is believed to date back to the 15th century. It contains illustrations of plants, astrological symbols, and mysterious text that has yet to be translated. Some researchers have suggested that the manuscript could contain medicinal insights, as it appears to depict various plants and herbs that may have been used for medicinal purposes in medieval times.\n\nHowever, without a clear understanding of the text contained in the manuscript, it is difficult to determine the exact medicinal insights that may be present. Some researchers have proposed that the illustrations of plants in the manuscript could be a form of early botanical knowledge, potentially containing information on herbal remedies and medicinal practices of the time.\n\nOverall, while the Voynich manuscript remains a fascinating and enigmatic text, its true medicinal insights remain unknown until the text can be successfully deciphered.', re

In [53]:
llm.invoke("What is Aetherfloris Ventus?")

AIMessage(content='Aetherfloris Ventus is a term that appears to be a combination of Latin words, translating to "ether flower wind" in English. It could be a creative or poetic description of a floral scent carried by the wind, or it could be a name given to a specific product or concept related to flowers and wind. Without more context, it is difficult to determine the specific meaning of Aetherfloris Ventus.', response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 16, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

## Load vector database from disk

In [54]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


db = FAISS.load_local("../faiss_index", 
                      OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small"), 
                      allow_dangerous_deserialization=True)

## Configure retriever
### Use the similarity search capabilities of a vector store to facilitate retrieval

In [55]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

## Preserve Conversation History 

In [56]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """Given the chat history and a recent user question \
generate a new standalone question \
that can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed or otherwise return it as is."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

retriever_with_history = create_history_aware_retriever(
    llm, retriever, prompt
)

## Perform question answering with chat history

In [57]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(retriever_with_history, question_answer_chain)

In [58]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Aetherfloris Ventus?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

print(ai_msg_1["answer"])

Aetherfloris Ventus is a celestial plant with petals lighter than air, appearing to float freely, nurtured by the breath of the winds and whispers of the clouds. Its nearly invisible stem dances with the breeze, leading the petals in a delicate ballet, inspiring myths and legends. In the realm of alchemy, its essence is treasured for bestowing lightness upon those who partake.


In [59]:
second_question = "What does a single drop of it do?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
print(ai_msg_2["answer"])

A single drop of Aetherfloris Ventus can lift spirits, freeing the mind from earthly burdens, encouraging thoughts to soar and dreams to take flight. Its essence, captured in rare vials, is believed to bestow the gift of lightness upon those who partake, inspiring a sense of weightlessness and freedom. The true nature of Aetherfloris Ventus eludes even dedicated seekers, remaining a mysterious treasure in the natural world.


In [60]:
third_question = "How does it compare to Noctis Umbraherba?"
ai_msg_3 = rag_chain.invoke({"input": third_question, "chat_history": chat_history})
print(ai_msg_3["answer"])

Aetherfloris Ventus is a celestial plant with light petals that appear to float freely, nurtured by wind and clouds, while Noctis Umbraherba is a shadowy plant thriving in darkness, with leaves that absorb shadows and blossoms blooming only at night. Aetherfloris Ventus embodies the essence of the sky, while Noctis Umbraherba is a plant shrouded in mystery, a whispered legend in botany.


In [61]:
fourth_question = "Do you think the Biological section of the Voynich Manuscript is important?"
ai_msg_4 = rag_chain.invoke({"input": fourth_question, "chat_history": chat_history})
print(ai_msg_4["answer"])

The Biological section of the Voynich Manuscript is crucial as it showcases a diversity of life forms, both creatures and plants, in a mysterious and enigmatic manner. It delves into intricate anatomical details, revealing complex and interconnected life systems that spark curiosity and exploration. The section not only demonstrates a blend of science and mystique but also provides a window into ancient perceptions of life, offering insights into how these beings and plants were understood and possibly utilized in various practices.
